# Functions

In [1]:
// Functions used to read data and simulations

## Calibration

In [2]:
TH1D* Calibrate_histo(TH1D* histo, Double_t CalibVal, TString NewTitle, Double_t Offset=0){
    Int_t NBins = histo->GetNbinsX();
    Int_t MCA_min = histo->GetBinCenter(1);
    Int_t MCA_max = histo->GetBinCenter(NBins);
    
    TH1D* Calib_histo = new TH1D(NewTitle, Form("%s_keVee", histo->GetTitle()), NBins, MCA_min*CalibVal, MCA_max*CalibVal);
    Int_t MCA_i;
    Double_t E_cal, MCA_i_content, MCA_i_Error;
    for(int i=0; i<NBins; i++){
        MCA_i = histo->GetBinCenter(i);
        MCA_i_content = histo->GetBinContent(i)*histo->GetBinWidth(i);
        MCA_i_Error = histo->GetBinError(i)*histo->GetBinWidth(i);
        E_cal = (MCA_i-Offset)*CalibVal;
        int Bin_Ecal = Calib_histo->FindBin(E_cal);
        Double_t prev_val_Ecal = Calib_histo->GetBinContent(Bin_Ecal);
        Double_t prev_err_Ecal = Calib_histo->GetBinError(Bin_Ecal);
        Calib_histo->SetBinContent(Bin_Ecal, prev_val_Ecal+MCA_i_content/Calib_histo->GetBinWidth(i));
        Calib_histo->SetBinError(Bin_Ecal, sqrt(prev_err_Ecal*prev_err_Ecal+(MCA_i_Error/Calib_histo->GetBinWidth(i))*(MCA_i_Error/Calib_histo->GetBinWidth(i))));
    }
    Calib_histo->GetXaxis()->SetTitle("Calibrated Energy [keVee]");
    Calib_histo->GetYaxis()->SetTitle("Differential rate (ev.d^{-1}.keVee^{-1})");

    return Calib_histo;
}

## Sigma

In [3]:
//Standard deviation of a vector
Double_t Sigma(vector<Double_t> Vector){
    int num_val = Vector.size();
    Double_t Mean=0;
    for (int i=0; i<num_val; i++){
        Mean+=Vector[i];
    }
    Mean= Mean/num_val;
    
    Double_t sigma=0;
    for (int i=0; i<num_val; i++){
        sigma+=(Vector[i]-Mean)*(Vector[i]-Mean);
    }
    sigma= sigma/(num_val-1);
    sigma = sqrt(sigma);
    return sigma;
}

In [4]:
//Smooth Step function
//--------------------------------------------------------------
double cdf(double *x, double *par){
    double mean=par[0];
    double sigma = par[1];
    
    double xx =x[0];
    return ROOT::Math::normal_cdf(xx, sigma, mean);
}

In [5]:
TH1D* Put_data_in_histo(TString filename, double real_time, TString Title){
    ifstream in;
    in.open(filename, ios::in);
    cout<<"\n*******************************************************"<<endl;
    cout<<filename<<endl;
    Float_t time=real_time/(24*3600.);
    
    Int_t x,y, MCA_chan;
    TH1D* h1 = new TH1D(Title,Title,512,-0.5,511.5);
    h1->GetXaxis()->SetTitle("Pulse Height [MCA a.u.]");
    h1->GetYaxis()->SetTitle("Counts per day per MCA channel");
    
    while (1) {
        in >> x >> y;
        if (!in.good()) break;
        
        h1->SetBinContent(x+1, y);
        h1->SetBinError(x+1, sqrt(y));
    }
    MCA_chan=h1->GetBinWidth(1);
    h1->Scale(1./(MCA_chan*time));
    return h1;
}

In [6]:
TH1D* Put_simu_in_histo_nq(TString filename, TString Title, int nBin, double Emin, double Emax, Double_t NumEv, Double_t Flux, Double_t MVthr=3., Double_t resol=1.){
    TFile* Infile = new TFile(filename);
    
    Float_t ECr, EMV;
    TRandom gen(0);
    
    Double_t Plane_surf = 40.*40.; //cm2
    
    Double_t Time = NumEv/(Plane_surf*Flux); //s
    
    TTree *InTree = (TTree*) Infile->Get("NTupleAlgoDir/Pri_Ntuple");
    Int_t entries= InTree->GetEntries();
    printf(" Entries:      %10d\n",entries);
    printf(" Generated primaries:  %.0f\n",NumEv);
    printf(" Equivalent time:      %.0f s (%.3f d)\n",Time,Time/86400.);

    TH1D *histo = new TH1D(Title,Title,nBin,Emin,Emax);
    histo->GetXaxis()->SetTitle("Energy [keVee]");
    histo->GetYaxis()->SetTitle("Counts.d^{-1}.keVee^{-1}");

    InTree->SetBranchAddress("EDep", &ECr);
    InTree->SetBranchAddress("EMV0", &EMV);
    for (Int_t pk=0; pk<entries; pk++) {
        InTree->GetEntry(pk);
        
        Float_t Ep = EMV;
        
        if (Ep>0){
            Float_t sp = 0.1008*sqrt(1./Ep*(1.+1./Ep))/2.3548;      // (%)
            Ep *= (1. + sp*gen.Gaus())*1000.;
            if (Ep < MVthr*1000) continue;
        }
        
        Float_t E = ECr*1000.;                            // keV
        Float_t Enew = gen.Gaus(E, resol*sqrt(E)); //Neutron peak assumed to be at 3500 keVee
        histo->Fill(Enew);
    }
    Double_t Ei = histo->GetBinWidth(1);
    histo->Scale(1./(Time/86400.)/Ei);
    
    return histo;
}

In [7]:
//This function applies a quenching factor
TH1D* Put_simu_in_histo(TString filename, TString Title, int nBin, double Emin, double Emax, Double_t NEvt, Double_t Flux =0.0134, Double_t MVthr=3., bool Coinc=true, Double_t resol = 0.07){                                                      // D08 <----------
    
    TFile* Nfile = new TFile(filename);

    Float_t ECr, EMV;
    TRandom gen(0);
    
    Double_t NTime = NEvt/40./40./Flux;     // s
    
    
    TTree *Nnp = (TTree*) Nfile->Get("NTupleAlgoDir/Pri_Ntuple");
    TTree *Nns = (TTree*) Nfile->Get("NTupleAlgoDir/Sec_Ntuple");

    Int_t Np_entries= Nnp->GetEntries();
    Int_t Ns_entries= Nns->GetEntries();
    printf("  Primary Entries:      %10d\n",Np_entries);
    printf("  Secondary Entries:    %10d\n",Ns_entries);
    printf("  Generated primaries:  %.0f\n",NEvt);
    printf("  Equivalent time:      %.0f s (%.3f d)\n",NTime,NTime/86400.);
    
    TH1D *neut = new TH1D(Title,Title,nBin,Emin,Emax);
    neut->GetXaxis()->SetTitle("Energy [keVee]");
    neut->GetYaxis()->SetTitle("Counts.d^{-1}.keVee^{-1}");

    Nnp->SetBranchAddress("EDep", &ECr);
    Nnp->SetBranchAddress("EMV0", &EMV);

    Float_t NpNSec,NpDelay, NsPDG,NsPidM,NsPid;
    Nnp->SetBranchAddress(  "NSec", &NpNSec);
    Nnp->SetBranchAddress( "Delay", &NpDelay);
    Nns->SetBranchAddress(   "PDG", &NsPDG);
    Nns->SetBranchAddress(  "PidM", &NsPidM);
    Nns->SetBranchAddress(   "Pid", &NsPid);

    // Calculate a constant q.f. from the position, on the gamma calibrated Energy scale, of the thermal neutron peak (expected at 4.78 MeVee)
    Float_t qf = 3500/4780.; //Quenching factor Emin + 326/512(Position of neutron peak/number of bins)*(Emax-Emin)/4780(Qvalue the neutron capture)
    
    // Loop on primary particles pk
    Int_t pk=0, sk=0, npCheck=0, nsCheck=0, isQF=0, nQuenched=0;
    
    while (pk < Np_entries) {
        
        // Loop on secondary particles sk (defined as those particles entering in the LiI(Eu) crystal)
        if (sk < Ns_entries) Nns->GetEntry(sk);
        
        // Get the "id" of the primary particles associated to the secondary particle sk
        Int_t id = ((Int_t) (NsPidM+0.5))*1000000 + ((Int_t) (NsPid+0.5));
        
         // if the primary particles is pk (the one under examination in primary loop), set isQF=1 in the cases when the
        // secondary particle sk is a neutron or antineutron
        // ==> i.e., if the secondary particles entering in the LiI(Eu) crystal is a neutron, the deposited energy is due
        // to a neutron interaction in LiI(Eu) detector and the quenching factor has to be applied (isQF=1, --> true).
        if(id==pk && sk < Ns_entries) {
            if (NsPDG==2112 || NsPDG==-2112) isQF=1;
            sk++;
            nsCheck++;
        } 
        
        // otherwise, no further secondary particles are associated to pk, I can fill the spectrum with the deposited
        // energy in LiI(Eu) detector (branch EDep in primary ntuple), applying or not the quenching factor
        else {
            Nnp->GetEntry(pk);
            // it is only a consinstency check (verify that the number of analysed secondary particles is the expected one)
            // for the pk primary particle under examination
            if (NpNSec != nsCheck) printf(" Primary entry %d: NSec error NSec=%d, nsCheck=%d\n", pk, (int) NpNSec, nsCheck);
    
            // It happens sometime that the primary particles is not associated to any secondary particles (NpNSec=0);
            // Almost always, it is due to very slow neutrons that after entering the LiI(Eu) crystal need long time before
            // having an interaction, and therefore are classified as new, delayed events (NpDelay>0) and loose their correlation
            // with the secondary particle ntuple. Also these events have to be classified as neutron events ==> isQF=1
            // (this point is not easy to explain, maybe we can have a short discussion about it)
            if (NpNSec==0 && NpDelay>0) isQF=1;        // Sicuramente neutroni
            if (isQF == 1) nQuenched++;
            
            // the E deposition in the MV is spread accounting for energy resolution resolution
            Float_t Ep = EMV;
        
            if (Ep>0){
                Float_t sp = 0.1008*sqrt(1./Ep*(1.+1./Ep))/2.3548;      // (%)
                Ep *= (1. + sp*gen.Gaus())*1000.;
            }
            
            if (Coinc){
                if (Ep > MVthr*1000) {
                    Float_t E = ECr*1000.;                                  // keV

                    if (isQF==1) E *= qf;
                    Double_t Enew = gen.Gaus(E, resol*sqrt(E)); //Neutron peak assumed to be at 3500 keVee                           // keV
                    neut->Fill(Enew);
                } // else printf("Ep = %f keV\n",EMV);
            }
            
            else {
                Float_t E = ECr*1000.;                                  // keV

                if (isQF==1) E *= qf;
                Double_t Enew = gen.Gaus(E, resol*sqrt(E)); //Neutron peak assumed to be at 3500 keVee                           // keV
                neut->Fill(Enew);
            }
            
            npCheck++;
            nsCheck=0;
            isQF=0;
            pk++;
        }
    }

    printf(" Nb of primary analysed: %d\n", npCheck);
    printf(" Nb of events quenched : %d\n", nQuenched);
    
    Double_t Ei = neut->GetBinWidth(1);
    neut->Scale(1./(NTime/86400.)/Ei);
    return neut;
}

In [8]:
// Residual histogram
TH1D* Residuals(TH1D* histo_data, TF1* Model, Double_t Min, Double_t Max, bool pc = true){
    TH1D* hRes = (TH1D*) histo_data->Clone();
    double Value_i, E_i, Diff_i, Error_i;
    double chi2=0;
    for (int i = 1; i<hRes->GetNbinsX(); i++){
        Value_i = hRes->GetBinContent(i);
        E_i = hRes->GetBinCenter(i);
        if (Value_i>0&&E_i>Min&&E_i<Max) {
            Diff_i = histo_data->GetBinContent(i)-Model->Eval(E_i);
            Error_i = histo_data->GetBinError(i);
            if (pc) {
                hRes->SetBinContent(i, Diff_i/Value_i*100);
                hRes->SetBinError(i, Error_i/Value_i*100);
            }
            else {
                hRes->SetBinContent(i, Diff_i/Error_i);
                hRes->SetBinError(i, 1);
            }
            chi2+=(Diff_i*Diff_i)/(Error_i*Error_i);
        }
        else 
        {hRes->SetBinContent(i, 0);
        hRes->SetBinError(i, 0);}
    }

    //cout<<"\nChi2/ndf = "<<chi2<<"/"<<Model->GetNDF()<<" = "<<chi2/Model->GetNDF()<<endl;
    hRes->SetLineColor(Model->GetLineColor());
    hRes->SetTitle("");
    if (pc) hRes->GetYaxis()->SetRangeUser(-100, 100);
    else hRes->GetYaxis()->SetRangeUser(-20, 20);
    return hRes;
}

In [9]:
// Residual histogram
TH1D* Residuals_histos(TH1D* histo_data, TH1D* Model, Double_t Min, Double_t Max, bool pc = true){
    TH1D* hRes = (TH1D*) histo_data->Clone();
    double Value_i, E_i, Diff_i, Error_i;
    double chi2=0;
    for (int i = 1; i<hRes->GetNbinsX(); i++){
        Value_i = hRes->GetBinContent(i);
        E_i = hRes->GetBinCenter(i);
        if (Value_i>0&&E_i>Min&&E_i<Max) {
            Diff_i = histo_data->GetBinContent(i)-Model->GetBinContent(i);
            Error_i = sqrt(histo_data->GetBinError(i)*histo_data->GetBinError(i)+Model->GetBinError(i)*Model->GetBinError(i));
            if (pc) {
                hRes->SetBinContent(i, Diff_i/Value_i*100);
                hRes->SetBinError(i, Error_i/Value_i*100);
            }
            else {
                hRes->SetBinContent(i, Diff_i/Error_i);
                hRes->SetBinError(i, 1);
            }
            chi2+=(Diff_i*Diff_i)/(Error_i*Error_i);
        }
        else {
            hRes->SetBinContent(i, 0);
            hRes->SetBinError(i, 0);}
    }

    cout<<"\nChi2/ndf = "<<chi2<<"/"<<(Max-Min)/Model->GetBinWidth(1)<<" = "<<chi2/((Max-Min)/Model->GetBinWidth(1))<<endl;
    hRes->SetLineColor(Model->GetLineColor());
    hRes->SetTitle("");
    if (pc) hRes->GetYaxis()->SetRangeUser(-100, 100);
    else hRes->GetYaxis()->SetRangeUser(-20, 20);
    return hRes;
}

In [10]:
%jsroot on
#include <TFitResult.h>

# Calibration from gammas

## Muons data: coincidence between LiI(Eu) and MV

In [11]:
TH1D* h_MuonCoinc = Put_data_in_histo("Measures@Saclay/2022-11-14_BS-A-06_RT_489309.69_LV_182.41_MuonCoinc.dat", 489309.69, "8inch at Saclay muons in coincidence");
h_MuonCoinc->Rebin(4);
h_MuonCoinc->Scale(1/4.);


*******************************************************
Measures@Saclay/2022-11-14_BS-A-06_RT_489309.69_LV_182.41_MuonCoinc.dat


In [12]:
double Muons(double *x, double *par){
    return par[0]*TMath::Landau(x[0], par[1], par[2]);
}

In [13]:
TF1* f_Muons = new TF1("f_Muons", Muons, 0, 5000, 3);

In [14]:
f_Muons->SetParameter(0, 10.);
f_Muons->SetParameter(1, 120.);
f_Muons->SetParameter(2, 50);
h_MuonCoinc->Fit("gaus", "", "", 80, 150);

double posMCA_Mu = f_Muons->GetMaximumX();
double ErrposMCA_Mu = f_Muons->GetMaximumX()*(f_Muons->GetParError(1)/f_Muons->GetParameter(1));

 FCN=17.1291 FROM MIGRAD    STATUS=CONVERGED      85 CALLS          86 TOTAL
                     EDM=1.10427e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.85487e+00   2.19881e-01   3.23179e-04   5.87591e-04
   2  Mean         1.15347e+02   9.08617e-01   1.88731e-03  -1.09710e-04
   3  Sigma        2.93206e+01   1.41898e+00   3.06500e-05   7.55083e-03


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [15]:
TCanvas* C8 = new TCanvas();
C8->Draw();
C8->SetLogy();
h_MuonCoinc->Draw();

## Cf source and "calibration"

In [16]:
TH1D* h_Cal_8inch = Put_data_in_histo("Measures@Saclay/2022-11-08_BS-A-06_RT_777.23_LV_776.57_Cf252.dat", 777.23, "8inch with Cf252 n source");
h_Cal_8inch->Rebin(4);
h_Cal_8inch->Scale(1/4.);


*******************************************************
Measures@Saclay/2022-11-08_BS-A-06_RT_777.23_LV_776.57_Cf252.dat


In [17]:
TF1* Neutron_peak_Gaus_MCA = new TF1("Neutron_peak_MCA", "[0]*TMath::Gaus(x, [1], [2])", 200, 500); //Gaussian function to fit the neutron calibration run
Neutron_peak_Gaus_MCA->SetParNames("Amplitude [Counts/MCA]", "#mu [MCA]", "#sigma [MCA]");
Neutron_peak_Gaus_MCA->SetParameters(10, 350, 40);
h_Cal_8inch->Fit("Neutron_peak_MCA", "R");

Double_t Cal_factor_8inch = 3500./Neutron_peak_Gaus_MCA->GetParameter(1); //Neutron peak assumed to be at 3500keVee
Double_t err_Cal_factor_8inch = (Neutron_peak_Gaus_MCA->GetParError(1))/(Neutron_peak_Gaus_MCA->GetParameter(1))*Cal_factor_8inch; //Neutron peak assumed to be at 3500keVee
Double_t Resol_NP = Neutron_peak_Gaus_MCA->GetParameter(2)/Neutron_peak_Gaus_MCA->GetParameter(1);

double posMCA_NP = Neutron_peak_Gaus_MCA->GetParameter(1);
double ErrposMCA_NP = Neutron_peak_Gaus_MCA->GetParError(1);

 FCN=98.4938 FROM MIGRAD    STATUS=CONVERGED     137 CALLS         138 TOTAL
                     EDM=2.74707e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   3.2 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Amplitude [Counts/MCA]   4.59487e+03   1.12886e+02   3.69985e+00   6.39847e-06
   2  #mu [MCA]    3.36165e+02   5.25966e-01   3.40273e-02   1.07750e-03
   3  #sigma [MCA]   2.66246e+01   4.18097e-01  -2.19165e-02   2.76336e-03


In [18]:
TCanvas* C8 = new TCanvas();
C8->Draw();
C8->SetLogy();
h_Cal_8inch->Draw();

cout<<"Calibration for the 8inch sphere = "<<Cal_factor_8inch<<" +- "<< err_Cal_factor_8inch<<" keV/MCA"<<endl;

Calibration for the 8inch sphere = 10.4115 +- 0.01629 keV/MCA


In [19]:
TH1D* hCal_MuonCoinc = Calibrate_histo(h_MuonCoinc, Cal_factor_8inch, Form("%s_keVee", h_MuonCoinc->GetTitle()));

In [20]:
int nbin = hCal_MuonCoinc->GetNbinsX();
double Emin = hCal_MuonCoinc->GetBinLowEdge(1);
double Emax = hCal_MuonCoinc->GetBinLowEdge(nbin)+hCal_MuonCoinc->GetBinWidth(nbin);
TH1D* h_SimuMuons_8inch = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 0., true, 4.);

  Primary Entries:          502660
  Secondary Entries:        567025
  Generated primaries:  960000000
  Equivalent time:      31578947 s (365.497 d)
 Nb of primary analysed: 502660
 Nb of events quenched : 13


In [21]:
TH1D* h_SimuMuons_8inch_1MeV = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 1., true, 0.0);
h_SimuMuons_8inch_1MeV->Fit("landau", "R0", "", 900, 2000);

TH1D* h_SimuMuons_8inch_5MeV = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 5., true, 0.0);
h_SimuMuons_8inch_5MeV->Fit("landau", "R0", "", 900, 2000);

TH1D* h_SimuMuons_8inch_7MeV = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 7., true, 0.0);
h_SimuMuons_8inch_7MeV->Fit("landau", "R0", "", 900, 2000);

TH1D* h_SimuMuons_8inch_0MeV = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 0., true, 0.0);
h_SimuMuons_8inch_0MeV->Fit("landau", "R0", "", 900, 2000);

TH1D* h_SimuMuons_9inch45deg = Put_simu_in_histo("Simulations/Atm_Muons/BS_D09/Analyzer_T_BS9_45deg.root", "9 inches tilt 45deg - atmospheric muons", nbin, Emin, Emax, 5.0e8, 0.019, 1., true, 0.0);
h_SimuMuons_9inch45deg->Fit("landau", "R0", "", 800, 1900);

TCanvas *C = new TCanvas();
C->Draw();
C->SetLogy();
h_SimuMuons_8inch_1MeV->GetYaxis()->SetRangeUser(1e-3, 10);
h_SimuMuons_8inch_1MeV->SetLineColor(kBlue);
h_SimuMuons_8inch_5MeV->SetLineColor(kRed);
h_SimuMuons_8inch_7MeV->SetLineColor(kMagenta);
h_SimuMuons_8inch_0MeV->SetLineColor(kBlack);
h_SimuMuons_8inch_1MeV->Draw("HIST");
h_SimuMuons_8inch_5MeV->Draw("HIST SAME");
h_SimuMuons_8inch_7MeV->Draw("HIST SAME");
h_SimuMuons_8inch_0MeV->Draw("HIST SAME");
h_SimuMuons_9inch45deg->Draw("HIST SAME");


h_SimuMuons_8inch_1MeV->SetTitle("");
TLegend* leg0 = new TLegend(0.6, 0.7, 0.89, 0.89);
leg0->AddEntry(h_SimuMuons_8inch_1MeV, "Coinc Muons thr 1MeV");
leg0->AddEntry(h_SimuMuons_8inch_5MeV, "Coinc Muons thr 5MeV");
leg0->AddEntry(h_SimuMuons_8inch_7MeV, "Coinc Muons thr 7MeV");
leg0->AddEntry(h_SimuMuons_8inch_0MeV, "Coinc Muons thr 0MeV");
leg0->AddEntry(h_SimuMuons_9inch45deg, "Coinc Muons thr 1MeV, 45deg");
leg0->Draw("SAME");
leg0->SetLineWidth(0);

  Primary Entries:          502660
  Secondary Entries:        567025
  Generated primaries:  960000000
  Equivalent time:      31578947 s (365.497 d)
 Nb of primary analysed: 502660
 Nb of events quenched : 13
 FCN=329.757 FROM MIGRAD    STATUS=CONVERGED     126 CALLS         127 TOTAL
                     EDM=2.82357e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.76951e+00   2.45954e-02   1.46135e-04  -1.96668e-02
   2  MPV          1.03134e+03   1.02660e+00   6.75376e-03   2.36602e-04
   3  Sigma        1.00453e+02   3.83814e-01   4.43231e-06   4.09704e-01
  Primary Entries:          502660
  Secondary Entries:        567025
  Generated primaries:  960000000
  Equivalent time:      31578947 s (365.497 d)
 Nb of primary analysed: 502660
 Nb of events quenched : 13
 FCN=261.789 FROM MIGRAD    STATUS=CONVERGED     130 CALLS

Error in <TNtuple::SetBranchAddress>: unknown branch -> EMV0
Warning in <Fit>: Fit data is empty 


In [22]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
hCal_MuonCoinc->Draw();
hCal_MuonCoinc->GetYaxis()->SetRangeUser(1e-3, 2);
hCal_MuonCoinc->GetXaxis()->SetRangeUser(0, 4580);
h_SimuMuons_8inch->SetLineColor(kRed);
h_SimuMuons_8inch->Draw("SAME");
gStyle->SetOptStat(0);
hCal_MuonCoinc->SetTitle("");

TLegend* leg0 = new TLegend(0.6, 0.7, 0.89, 0.89);
leg0->AddEntry(hCal_MuonCoinc, "Data");
leg0->AddEntry(h_SimuMuons_8inch, "Simulation");
leg0->Draw("SAME");
leg0->SetLineWidth(0);

In [23]:
f_Muons->SetParameter(0, 1.);
f_Muons->SetParameter(1, 1000.);
f_Muons->SetParameter(2, 500);
h_SimuMuons_8inch->Fit("gaus", "", "", 800, 1300);

double posE_Mu = f_Muons->GetMaximumX();
double ErrposE_Mu = f_Muons->GetMaximumX()*(f_Muons->GetParError(1)/f_Muons->GetParameter(1));

 FCN=71.8458 FROM MIGRAD    STATUS=CONVERGED      96 CALLS          97 TOTAL
                     EDM=2.19354e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.13606e+00   3.91445e-03   1.13108e-05  -1.23747e-01
   2  Mean         1.04607e+03   1.05970e+00   4.40598e-03  -1.12361e-04
   3  Sigma        2.42710e+02   2.00401e+00   1.12509e-05   5.55869e-02


In [24]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_SimuMuons_8inch->GetYaxis()->SetRangeUser(1e-3, 2);
h_SimuMuons_8inch->SetLineColor(kRed);
h_SimuMuons_8inch->Draw("E0");

## Gamma sources

In [25]:
double Compton_Edge(double *x, double *par){
    double par_cdf[2] = {par[1], par[2]};
    return par[0]*(1-cdf(x, par_cdf));
}

In [26]:
TF1* f_compton_edge = new TF1("f_compton_edge", Compton_Edge, 0, 200, 3);

In [27]:
TH1D* h_8inch_Co60 = Put_data_in_histo("Measures@Saclay/2022-11-08_BS-A-06_RT_2114.05_LV_2112.16_Co60.dat", 2114.05, "8inch at Saclay - Co60 source");
h_8inch_Co60->Rebin(4);
h_8inch_Co60->Scale(1/4.);


*******************************************************
Measures@Saclay/2022-11-08_BS-A-06_RT_2114.05_LV_2112.16_Co60.dat


In [28]:
f_compton_edge->SetParameter(0, 2000);
f_compton_edge->SetParameter(1, 160);
f_compton_edge->SetParameter(2, 50);
h_8inch_Co60->Fit(f_compton_edge, "", "", 100, 260);

double posMCA_Co60 = f_compton_edge->GetParameter(1);
double ErrposMCA_Co60 = f_compton_edge->GetParError(1);

 FCN=37.1958 FROM MIGRAD    STATUS=CONVERGED     105 CALLS         106 TOTAL
                     EDM=7.48414e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.51036e+03   2.08208e+02   1.64452e-01   7.80624e-07
   2  p1           1.32039e+02   3.62258e+00   2.23301e-03   5.67967e-05
   3  p2           2.95542e+01   1.87139e+00   2.15577e-03   8.70207e-05


In [29]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_8inch_Co60->GetXaxis()->SetRangeUser(0, 250);
h_8inch_Co60->Draw("E0");

### Co60

In [63]:
TH1D* h_SimCo60_8inch = Put_simu_in_histo("Simulations/Co60/Analyzer_T_run01-20.root", "8 inches Co60", 128, 0., 5000., 3e8, 10240./(40*40), 6., false, 4.);

  Primary Entries:          599184
  Secondary Entries:             0
  Generated primaries:  300000000
  Equivalent time:      29297 s (0.339 d)
 Nb of primary analysed: 599184
 Nb of events quenched : 0


In [64]:
TH1D* h_SimCo60_8inch_noresol = Put_simu_in_histo("Simulations/Co60/Analyzer_T_run01-20.root", "8 inches Co60 no resol", 128, 0., 5000., 3e8, 10240./(40*40), 6., false, 0.0);

  Primary Entries:          599184
  Secondary Entries:             0
  Generated primaries:  300000000
  Equivalent time:      29297 s (0.339 d)
 Nb of primary analysed: 599184
 Nb of events quenched : 0


In [56]:
f_compton_edge->SetParameter(0, 200);
f_compton_edge->SetParameter(1, 1300);
f_compton_edge->SetParameter(2, 500);
f_compton_edge->SetParLimits(2, 0, 3000);
h_SimCo60_8inch->Fit(f_compton_edge, "", "", 1100, 1800);

double posE_Co60 = f_compton_edge->GetParameter(1);
double ErrposE_Co60 = f_compton_edge->GetParError(1);

 FCN=6.4142 FROM MIGRAD    STATUS=CONVERGED     165 CALLS         166 TOTAL
                     EDM=1.13781e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.65065e+02   3.05777e+01   3.87485e-03  -1.49312e-04
   2  p1           1.09931e+03   2.77461e+01   2.51054e-03  -2.08638e-04
   3  p2           2.23930e+02   9.28964e+00   2.67815e-06  -1.26213e-01


In [81]:
c1 = new TCanvas("c1", "c1", 1000, 700);
c1->Draw();

gStyle->SetCanvasPreferGL(kTRUE);
gPad->SetRightMargin(0.03);
gPad->SetTopMargin(0.02);
gPad->SetBottomMargin(0.1);
//gPad->SetGridx();
//gPad->SetGridy();
gPad->SetLogy();
gStyle->SetOptStat(0);

h_SimCo60_8inch->GetXaxis()->SetRangeUser(0, 2000);
h_SimCo60_8inch->Draw("HIST");
h_SimCo60_8inch->SetTitle("");
h_SimCo60_8inch->GetXaxis()->SetTitleOffset(1.2);

h_SimCo60_8inch_noresol->SetLineColor(kRed);
h_SimCo60_8inch_noresol->Draw("HIST SAME");

TLegend* leg0 = new TLegend(0.6, 0.85, 0.96, 0.96);
leg0->AddEntry(h_SimCo60_8inch, "with LiI(Eu) resolution");
leg0->AddEntry(h_SimCo60_8inch_noresol, "No resolution");
leg0->Draw("SAME");
leg0->SetLineWidth(0);

c1->SaveAs("/Users/cg264002/research/Redaction_These/Figures/Chap3/Neutrons/Simulation_w_wo_resolution.png");
//h_SimCo60_8inch->SetLineColor(kRed);

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file /Users/cg264002/research/Redaction_These/Figures/Chap3/Neutrons/Simulation_w_wo_resolution.png has been created


### Ba133

In [33]:
TH1D* h_Ba133 = Put_data_in_histo("Measures@Saclay/2022-11-08_noBS_RT_302.42_LV_301.89_Ba133.dat", 302.42, " no Sphere Saclay - Ba133 source");


*******************************************************
Measures@Saclay/2022-11-08_noBS_RT_302.42_LV_301.89_Ba133.dat


In [34]:
f_compton_edge->SetParameter(0, 2e5);
f_compton_edge->SetParameter(1, 50);
f_compton_edge->SetParameter(2, 20);
h_Ba133->Fit(f_compton_edge, "", "", 50,100);

double posMCA_Ba133 = f_compton_edge->GetParameter(1);
double ErrposMCA_Ba133 = f_compton_edge->GetParError(1);

 FCN=26.5811 FROM MIGRAD    STATUS=CONVERGED     136 CALLS         137 TOTAL
                     EDM=2.12905e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.87006e+05   7.60564e+04   1.02875e+01  -6.28876e-09
   2  p1           4.81678e+01   2.24666e+00   2.90915e-04  -7.87899e-04
   3  p2           8.22784e+00   7.19635e-01   1.50458e-06  -2.69248e-01


In [35]:
TCanvas* C = new TCanvas();

C->Draw();
C->SetLogy();
h_Ba133->GetYaxis()->SetRangeUser(0, 1e6);
h_Ba133->GetXaxis()->SetRangeUser(0, 100);
h_Ba133->Draw("E0");

In [36]:
TH1D* h_SimBa133 = Put_simu_in_histo("Simulations/Ba133/Analyzer_PT_LIEu_Ba133_2.root", "Ba133", 512, 0., 5000., 10e7, 0.6*290000./(40*40), 6., false, 4.);

  Primary Entries:          517445
  Secondary Entries:             0
  Generated primaries:  100000000
  Equivalent time:      575 s (0.007 d)
 Nb of primary analysed: 517445
 Nb of events quenched : 0


In [37]:
f_compton_edge->SetParameter(0, 20000);
f_compton_edge->SetParameter(1, 450);
f_compton_edge->SetParameter(2, 100);
h_SimBa133->Fit(f_compton_edge, "", "", 400, 650);

double posE_Ba133 = f_compton_edge->GetParameter(1);
double ErrposE_Ba133 = f_compton_edge->GetParError(1);

 FCN=25.8056 FROM MIGRAD    STATUS=CONVERGED     150 CALLS         151 TOTAL
                     EDM=1.44155e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.11755e+04   3.48184e+03   8.31620e-01  -1.55659e-06
   2  p1           4.01390e+02   1.26755e+01   2.17241e-03  -6.03371e-04
   3  p2           7.22823e+01   4.17510e+00   3.93046e-06  -2.67222e-01


In [38]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_SimBa133->Draw("E0");
h_SimBa133->GetXaxis()->SetRangeUser(0, 800);
h_SimBa133->SetLineColor(kRed);

## Calib plot

In [39]:
double Cal_factors[4] = {posMCA_Ba133, posMCA_Mu, posMCA_Co60, posMCA_NP};
double ErrCal_factors[4] = {ErrposMCA_Ba133, ErrposMCA_Mu, ErrposMCA_Co60, ErrposMCA_NP};
double Energies[4] = {posE_Ba133, posE_Mu, posE_Co60, 3000};
double ErrEnergies[4] = {ErrposE_Ba133, ErrposE_Mu*2, ErrposE_Co60, 400};
TGraphErrors* gCal_factors = new TGraphErrors(4, Cal_factors, Energies, ErrCal_factors, ErrEnergies);

In [40]:
c1 = new TCanvas("c1", "c1", 1000, 700);
c1->Draw();

gStyle->SetCanvasPreferGL(kTRUE);
gPad->SetRightMargin(0.03);
gPad->SetTopMargin(0.02);
gPad->SetBottomMargin(0.1);
gPad->SetGridx();
gPad->SetGridy();
gStyle->SetOptStat(0);

gCal_factors->SetMarkerStyle(21);
gCal_factors->SetMarkerSize(2);
gCal_factors->SetMarkerColor(kRed);

gCal_factors->Draw("AP");

TF1* Gain_value = new TF1("Gain_value", "pol1", 0, 4000);
gCal_factors->SetTitle("");

gCal_factors->Fit(Gain_value, "RN");
Gain_value->Draw("SAME");
Gain_value->SetLineColor(kBlue);

Double_t Offset = Gain_value->GetParameter(0);
Double_t Gain = Gain_value->GetParameter(1);

TLatex *latexBa = new TLatex(gCal_factors->GetX()[0]+5, gCal_factors->GetY()[0]-100,"^{133}Ba");
latexBa->SetTextSize(0.04);
latexBa->Draw("SAME");

TLatex *latexCo = new TLatex(gCal_factors->GetX()[1]+5, gCal_factors->GetY()[1]-100,"^{60}Co");
latexCo->SetTextSize(0.04);
latexCo->Draw("SAME");

TLatex *latexMu = new TLatex(gCal_factors->GetX()[2]+8, gCal_factors->GetY()[2]-100,"#mu");
latexMu->SetTextSize(0.04);
latexMu->Draw("SAME");

TLatex *latexN = new TLatex(gCal_factors->GetX()[3]+5, gCal_factors->GetY()[3]+100,"n");
latexN->SetTextSize(0.04);
latexN->Draw("SAME");

gCal_factors->GetYaxis()->SetTitle("Expected Energy [keV]");
gCal_factors->GetXaxis()->SetTitle("Pulse Height [MCA]");

c1->SaveAs("/Users/cg264002/research/Redaction_These/Figures/Chap3/Neutrons/BS_energy_calibration.pdf");

 FCN=0.0118271 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=3.22031e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -2.67780e+01   4.15090e+01   9.82400e-03   2.84522e-07
   2  p1           8.88184e+00   5.01392e-01   1.18658e-04  -2.86849e-05


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file /Users/cg264002/research/Redaction_These/Figures/Chap3/Neutrons/BS_energy_calibration.pdf has been created


In [41]:
4*sqrt(2903)

(double) 215.51798


## Calibrate data

In [42]:
TH1D* hReCal_MuonCoinc = Calibrate_histo(h_MuonCoinc, 1./Gain, Form("%s_Cal_fit", hCal_MuonCoinc->GetTitle()), Offset);

int nbin = hReCal_MuonCoinc->GetNbinsX();
double Emin = hReCal_MuonCoinc->GetBinLowEdge(1);
double Emax = hReCal_MuonCoinc->GetBinLowEdge(nbin)+hCal_MuonCoinc->GetBinWidth(nbin);
TH1D* h_SimuMuons_8inch = Put_simu_in_histo("../BS_BkgModel/Atm_Muons/BS_D08/Analyzer_T_run01-12.root", "8 inches atmospheric muons", nbin, Emin, Emax, 9.60e8, 0.019, 0., true, 0.08);

  Primary Entries:          502660
  Secondary Entries:        567025
  Generated primaries:  960000000
  Equivalent time:      31578947 s (365.497 d)
 Nb of primary analysed: 502660
 Nb of events quenched : 13


In [43]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
hReCal_MuonCoinc->GetXaxis()->SetRangeUser(0, 4580);
hReCal_MuonCoinc->Draw();
hReCal_MuonCoinc->GetYaxis()->SetRangeUser(1e-3, 2);
h_SimuMuons_8inch->SetLineColor(kRed);
h_SimuMuons_8inch->Draw("SAME");

hReCal_MuonCoinc->SetTitle("");
gStyle->SetOptStat(0);

TLegend* leg0 = new TLegend(0.6, 0.7, 0.89, 0.89);
leg0->AddEntry(hCal_MuonCoinc, "Data (recal)");
leg0->AddEntry(h_SimuMuons_8inch, "Simulation");
leg0->Draw("SAME");
leg0->SetLineWidth(0);

In [44]:
TH1D* hCal_8inch_Co60 = Calibrate_histo(h_8inch_Co60, 1./Gain, Form("%s_keVee", h_8inch_Co60->GetTitle()), Offset);
int nbin = hCal_8inch_Co60->GetNbinsX();
double Emin = hCal_8inch_Co60->GetBinLowEdge(1);
double Emax = hCal_8inch_Co60->GetBinLowEdge(nbin)+hCal_8inch_Co60->GetBinWidth(nbin);
TH1D* h_SimCo60_8inch = Put_simu_in_histo("Simulations/Co60/Analyzer_T_run01-20.root", "8 inches Co60", nbin, Emin, Emax, 3e8, 10240./(40*40), 6., false, 0.08);

  Primary Entries:          599184
  Secondary Entries:             0
  Generated primaries:  300000000
  Equivalent time:      29297 s (0.339 d)
 Nb of primary analysed: 599184
 Nb of events quenched : 0


In [45]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
hCal_8inch_Co60->Draw("E0");
hCal_8inch_Co60->GetXaxis()->SetRangeUser(0, 2000);
h_SimCo60_8inch->Draw("SAME");
h_SimCo60_8inch->SetLineColor(kRed);


TCanvas* C2 = new TCanvas("cRes_Co60_8inch", "cRes_Co60_8inch", C->GetWw(), 0.6*C->GetWh());
C2->Draw();
gPad->SetMargin(0.1,0.1,0.2,0.1);

TH1D* hRes_Co60_8inch=Residuals_histos(hCal_8inch_Co60, h_SimCo60_8inch, 300, 1800, true);
hRes_Co60_8inch->GetYaxis()->SetTitle("Residuals [%]");
hRes_Co60_8inch->SetTitle("");
hRes_Co60_8inch->GetYaxis()->SetTitleSize(2*hCal_8inch_Co60->GetYaxis()->GetTitleSize());
hRes_Co60_8inch->GetXaxis()->SetTitleSize(2*hCal_8inch_Co60->GetXaxis()->GetTitleSize());
hRes_Co60_8inch->GetYaxis()->SetLabelSize(2*hCal_8inch_Co60->GetYaxis()->GetLabelSize());
hRes_Co60_8inch->GetXaxis()->SetLabelSize(2*hCal_8inch_Co60->GetXaxis()->GetLabelSize());
hRes_Co60_8inch->Draw("E0");
hRes_Co60_8inch->GetXaxis()->SetRangeUser(0, 2000);

h_1sig = (TH1D*) hCal_8inch_Co60->Clone();
for (int i =1; i<=h_1sig->GetNbinsX(); i++){
    h_1sig->SetBinContent(i, 0);
    h_1sig->SetBinError(i, 10);
}
h_1sig->SetLineColorAlpha(kOrange+1, 0.2);
h_1sig->SetMarkerColorAlpha(kOrange+1, 0.2);
h_1sig->SetFillColorAlpha(kOrange+1, 0.2);
h_1sig->Draw("E3SAME");


Chi2/ndf = 0/3356.91 = 0


In [46]:
TH1D* hCal_Ba133 = Calibrate_histo(h_Ba133, 1./Gain, Form("%s_keVee", h_Ba133->GetTitle()), Offset);
int nbin = hCal_Ba133->GetNbinsX();
double Emin = hCal_Ba133->GetBinLowEdge(1);
double Emax = hCal_Ba133->GetBinLowEdge(nbin)+hCal_Ba133->GetBinWidth(nbin);
TH1D* h_SimBa133 = Put_simu_in_histo("Simulations/Ba133/Analyzer_PT_LIEu_Ba133_2.root", "Ba133", nbin, Emin, Emax, 10e7, 0.6*290000./(40*40), 6., false);

  Primary Entries:          517445
  Secondary Entries:             0
  Generated primaries:  100000000
  Equivalent time:      575 s (0.007 d)
 Nb of primary analysed: 517445
 Nb of events quenched : 0


In [47]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
hCal_Ba133->Draw("E0");
hCal_Ba133->GetXaxis()->SetRangeUser(0, 2000);
hCal_Ba133->GetYaxis()->SetRangeUser(0, 1e6);
//h_SimBa133->Draw("SAME");
//h_SimBa133->SetLineColor(kRed);


TCanvas* C2 = new TCanvas("cRes_Ba133", "cRes_Ba133", C->GetWw(), 0.6*C->GetWh());
C2->Draw();
gPad->SetMargin(0.1,0.1,0.2,0.1);

TH1D* hRes_Ba133=Residuals_histos(hCal_Ba133, h_SimBa133, 300, 800, true);
hRes_Ba133->GetYaxis()->SetTitle("Residuals [%]");
hRes_Ba133->SetTitle("");
hRes_Ba133->GetYaxis()->SetTitleSize(1.8*hCal_Ba133->GetYaxis()->GetTitleSize());
hRes_Ba133->GetXaxis()->SetTitleSize(1.8*hCal_Ba133->GetXaxis()->GetTitleSize());
hRes_Ba133->GetYaxis()->SetLabelSize(1.8*hCal_Ba133->GetYaxis()->GetLabelSize());
hRes_Ba133->GetXaxis()->SetLabelSize(1.8*hCal_Ba133->GetXaxis()->GetLabelSize());
hRes_Ba133->Draw("E0");
hRes_Ba133->GetXaxis()->SetRangeUser(0, 2000);

h_1sig->Draw("E3SAME");


Chi2/ndf = 0/4449.61 = 0


In [48]:
TH1D* h_SimCo60_8inch_noresol = Put_simu_in_histo("Simulations/Co60/Analyzer_T_run01-20.root", "8 inches Co60 no resol", 128, 0., 5000., 3e8, 10240./(40*40), 6., false, 0.0);

  Primary Entries:          599184
  Secondary Entries:             0
  Generated primaries:  300000000
  Equivalent time:      29297 s (0.339 d)
 Nb of primary analysed: 599184
 Nb of events quenched : 0


In [49]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_SimCo60_8inch_noresol->Draw("HIST");

In [50]:
TH1D* h_SimBa133_noresol = Put_simu_in_histo("Simulations/Ba133/Analyzer_PT_LIEu_Ba133_2.root", "Ba133 no resol", 512, 0., 5000., 10e7, 0.6*290000./(40*40), 6., false, 0.);

  Primary Entries:          517445
  Secondary Entries:             0
  Generated primaries:  100000000
  Equivalent time:      575 s (0.007 d)
 Nb of primary analysed: 517445
 Nb of events quenched : 0


In [51]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_SimBa133_noresol->Draw("HIST");

In [52]:
TH1D* h_8inch_Co60_nosphere = Put_data_in_histo("Measures@Saclay/2022-11-28_noBS_RT_349.92_LV_349.41_Co60.dat", 349.92, " Saclay - Co60 source no sphere");
h_8inch_Co60_nosphere->Rebin(4);
h_8inch_Co60_nosphere->Scale(1/4.);


*******************************************************
Measures@Saclay/2022-11-28_noBS_RT_349.92_LV_349.41_Co60.dat


In [53]:
TCanvas* C = new TCanvas();
C->Draw();
C->SetLogy();
h_8inch_Co60->Draw("Hist");
h_8inch_Co60_nosphere->Draw("HIST same");
h_8inch_Co60_nosphere->Scale(h_8inch_Co60->GetBinContent(h_8inch_Co60->FindBin(100))/h_8inch_Co60_nosphere->GetBinContent(h_8inch_Co60_nosphere->FindBin(100)));
h_8inch_Co60->SetLineColor(kRed);

TLegend* leg0 = new TLegend(0.6, 0.7, 0.89, 0.89);
leg0->AddEntry(h_8inch_Co60, "With 8inch sphere");
leg0->AddEntry(h_8inch_Co60_nosphere, "No spheres (rescaled)");
leg0->Draw("SAME");
leg0->SetLineWidth(0);